# Load Dataset

In [ ]:

from tqdm import tqdm
from tqdm import trange

Index = []
Name = {}
Type = {}
Label = {}

train_Index = []
test_Index = []

count=0
flag=False
with open('./node.dat','r') as file:
    for line in file:
        count+=1
        temp = line[:-1].split('\t')

        if len(temp)!=4 and flag==False:
            print(temp)
            print(count)
            flag=True
            
        Index.append(int(temp[0]))
        Name[int(temp[0])] = temp[1]
        Type[int(temp[0])] = int(temp[2])

In [ ]:
with open('./label.dat','r') as file:
    for line in file:
        temp = line[:-1].split('\t')
        train_Index.append(int(temp[0]))
        Label[int(temp[0])] = temp[3].split(',')
        Label[int(temp[0])].sort()


In [ ]:
with open('./label.dat.test','r') as file:
    for line in file:
        temp = line[:-1].split('\t')
        test_Index.append(int(temp[0]))
        Label[int(temp[0])] = temp[3].split(',')
        Label[int(temp[0])].sort()


# Calculate # of Node Tokens

In [ ]:
from transformers import AutoTokenizer
model_name = "Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def calculate_token_num(text):
    encoded = tokenizer.encode(text)
    return len(encoded)

node_token_num = []
node_token_num_movie = []
for i in trange(len(Index)):
    text = Name[i]
    node_token_num.append(calculate_token_num(text))
    if Type[i] == 0: node_token_num_movie.append(calculate_token_num(text))
print(len(node_token_num))

In [ ]:
import numpy as np
tokens = np.array(node_token_num)
print("All node:")
print(f"Max #node tokens: {tokens.max()}")
print(f"Min #node tokens: {tokens.min()}")
print(f"Avg. #node tokens: {tokens.mean()}")

tokens_movie = np.array(node_token_num_movie)
print("Movie node:")
print(f"Max #node tokens: {tokens_movie.max()}")
print(f"Min #node tokens: {tokens_movie.min()}")
print(f"Avg. #node tokens: {tokens_movie.mean()}")


# Calculate # of Node Degrees

In [ ]:
Graph=[[] for i in range(len(Name))]
Self_to_Self_node_count = 0
Edge_Num = 0
with open('./link.dat','r') as file:
    for line in file:  
        start,end,e_type,_=line[:-1].split('\t')
        Edge_Num+=1
        if int(end) == int(start): Self_to_Self_node_count+=1
        Graph[int(start)].append(int(end))
    print("Self-Self:", Self_to_Self_node_count)
    print("# Nodes:", len(Graph))
    print("# Edges:", Edge_Num)

In [ ]:
import numpy as np
from tqdm import trange

node_degree_num = []
node_degree_num_movie = []
count=0
for i in trange(len(Graph)): 
    node_degree_num.append(len(Graph[i]))
    
    if len(Graph[i])==0: count+=1
    
    if Type[i]==0: node_degree_num_movie.append(len(Graph[i]))
    

degrees_before = np.array(node_degree_num)
degrees_before_movie = np.array(node_degree_num_movie)

print("All node:")
print(f"Max #node degrees_before: {degrees_before.max()}")
print(f"Min #node degrees_before: {degrees_before.min()}")
print(f"Avg. #node degrees_before: {degrees_before.mean()}")

print(f"#node degree=0: {count}")
print(f"#node degree=0/#total node: {count/len(Graph)}")

print("Movie node:")
print(f"Max #node degrees_before: {degrees_before_movie.max()}")
print(f"Min #node degrees_before: {degrees_before_movie.min()}")
print(f"Avg. #node degrees_before: {degrees_before_movie.mean()}")


In [ ]:
import numpy as np

degrees = np.ceil(np.log1p(degrees_before))

degrees_movie = np.ceil(np.log1p(degrees_before_movie))

print("All node:")
print(f"Max #node degrees: {degrees.max()}")
print(f"Min #node degrees: {degrees.min()}")
print(f"Avg. #node degrees: {degrees.mean()}")

print("Movie node:")
print(f"Max #node degrees: {degrees_movie.max()}")
print(f"Min #node degrees: {degrees_movie.min()}")
print(f"Avg. #node degrees: {degrees_movie.mean()}")


# Calculate Node H

In [ ]:
H = tokens*degrees # H = T * [log(D+1)]
H_movie = tokens_movie*degrees_movie

print("# Nodes:", len(H))
print(f"Max node H: {H.max()}")
print(f"Min node H: {H.min()}")
print(f"Avg. node H: {H.mean()}")
print(f"Sum node H: {H.sum()}")
print("------------------------")
print("# Nodes:", len(H_movie))
print(f"Max node H: {H_movie.max()}")
print(f"Min node H: {H_movie.min()}")
print(f"Avg. node H: {H_movie.mean()}")
print(f"Sum node H: {H_movie.sum()}")


# Generate Node's one-hop Neighbors and Random Walks


In [ ]:
train_nodes=train_Index

Neighs = {}
for node in train_nodes: Neighs[node] = []
Avg_degree = degrees_before_movie.mean()

for i,central_node in enumerate(train_nodes):

    neighs = Graph[central_node] # Neighbors
    central_node_H = H[central_node]/2 # H*/2

    neighs2H = {}
    for j in neighs: neighs2H[j] = H[j]
    sorted_neighs2H = dict(sorted(neighs2H.items(), key=lambda item: item[1], reverse=True))
    
    # Obtain central node's Neighbor (#Neighbors <= Avg. #Degree)
    Neigh_temp = []
    for neigh, neigh_H in sorted_neighs2H.items(): 
        Neigh_temp.append(neigh)
    if len(Neigh_temp) >= int(Avg_degree): 
        Neighs[central_node]= Neigh_temp[:int(Avg_degree)]
        for neigh in range(int(Avg_degree),len(Neigh_temp)):
             if sorted_neighs2H[Neigh_temp[neigh]] >= central_node_H: 
                 Neighs[central_node].append(Neigh_temp[neigh])
    else:  Neighs[central_node] = Neigh_temp
    

count =0
avg_count=0
for i,central_node in enumerate(train_nodes):
    if len(Neighs[central_node])==0: count+=1
    avg_count+=len(Neighs[central_node])
print(f"#Central Node's obtained Neighbor=0: {count}")
print(f"#Central Node's obtained Neighbor=0: {count/len(train_nodes)*100}%")
print(f"Avg. #Central Node's obtained Neighbor: {avg_count/len(train_nodes)}")
print(f"Avg. #Central Node's obtained Neighbor (#/=0): {avg_count/(len(train_nodes)-count)}")


In [ ]:

import random
RWs = {}
for node in train_nodes: RWs[node] = []
start_idx = train_nodes 

rand=random.Random()
max_walk_num = int(Avg_degree)+int(degrees_before_movie.min())
alpha=0.15
count=0
path_length=10000
for line in tqdm(range(len(start_idx))):
    central_node = start_idx[line]
    

    if len(Graph[central_node])==0: 
        continue
    
    if len(Neighs[central_node]) < max_walk_num: 
        walk_num = max_walk_num - len(Neighs[central_node])
    else: continue

    node_paths = []
    while len(node_paths) < walk_num:

        temp_path=[]
        start = central_node
        temp_path.append(start)
        
        if len(Graph[temp_path[-1]]) == 0: 
            break 
        for i in range(path_length):
            cur = temp_path[-1]
            

            if (len(Graph[cur])>0) and rand.random()>=alpha:
                next_ = rand.choice(Graph[cur])
          
                while(rand.random()>=alpha): 
                    next_ = rand.choice(Graph[cur])
                   
                temp_path.append(next_)
            else: 
                break
        if len(temp_path)>2 and len(temp_path)<=14: 
            node_paths.append(temp_path)
    RWs[central_node] = node_paths

In [ ]:
count =0
avg_count=0
for i,central_node in enumerate(train_nodes):
    if len(RWs[central_node])==0: 

        count+=1
    avg_count+=len(RWs[central_node])
print(f"#Central Node's obtained RWs=0: {count}")
print(f"#Central Node's obtained RWs=0: {count/len(train_nodes)*100}%")
print(f"Avg. #Central Node's obtained RWs: {avg_count/len(train_nodes)}")
print(f"Avg. #Central Node's obtained RWs (#/=0): {avg_count/(len(train_nodes)-count)}")

# Generate Node Classification Instruction

In [ ]:
# compact graph description
def get_subgraph(central_node, Neighs, RWs):
    
    Input =  "The compact graph description of this MOVIE is listed as follows:\n"
    Input += "Title: {" + Name[central_node] + "}"
    
    if len(Neighs)==0 and len(RWs)==0: 
        print(central_node)
        return Input
    else: Input += " "
    
    #---------------------------------------------------------------------

    input_graph = "Ego graph nodes: {"
    
    graph_MOVIE = []
    graph_DIRECTOR = []
    graph_ACTOR = []
    graph_KEYWORD = []
    # graph_MOVIE.append(central_node)
    

    for node in Neighs: 
        if node == central_node: continue
        
        if Type[node] == 0: graph_MOVIE.append(node)
        elif Type[node] == 1: graph_DIRECTOR.append(node)
        elif Type[node] == 2: graph_ACTOR.append(node)
        elif Type[node] == 3: graph_KEYWORD.append(node)
    
    # 加入RW中node信息:
    for path in RWs:
        for node in path:
            if node == central_node: continue
        
            if Type[node] == 0: graph_MOVIE.append(node)
            elif Type[node] == 1: graph_DIRECTOR.append(node)
            elif Type[node] == 2: graph_ACTOR.append(node)
            elif Type[node] == 3: graph_KEYWORD.append(node)
    
    tmp = []
    tmp.append(central_node)
    
    graph_MOVIE = tmp + list(set(graph_MOVIE))
    graph_DIRECTOR = list(set(graph_DIRECTOR))
    graph_ACTOR = list(set(graph_ACTOR))
    graph_KEYWORD = list(set(graph_KEYWORD))
    

    node2index = {}
    count = 0
    tmp_count = 0
    if len(graph_MOVIE) >0:
        count += 1
        input_graph += "MOVIE: ["
        for j,node in enumerate(graph_MOVIE):
            if j != (len(graph_MOVIE)-1): input_graph += ("("+str(tmp_count+j+1)+") \'" + Name[node]+"\', ")
            else: input_graph += ("("+str(tmp_count+j+1)+") \'" + Name[node]+"\']}\n")
            node2index[node]=str(tmp_count+j+1)
        tmp_count+=len(graph_MOVIE)
    
    if len(graph_DIRECTOR) >0:
        count += 1
        input_graph += " DIRECTOR: ["
        for j,node in enumerate(graph_DIRECTOR):
            if j != (len(graph_DIRECTOR)-1): input_graph += ("("+str(tmp_count+j+1)+") \'" + Name[node]+"\', ")
            else: input_graph += ("("+str(tmp_count+j+1)+") \'" + Name[node]+"\']}\n")
            node2index[node]=str(tmp_count+j+1)
        tmp_count+=len(graph_DIRECTOR)
    
    if len(graph_ACTOR) >0:
        count += 1
        input_graph += " ACTOR: ["
        for j,node in enumerate(graph_ACTOR):
            if j != (len(graph_ACTOR)-1): input_graph += ("("+str(tmp_count+j+1)+") \'" + Name[node]+"\', ")
            else: input_graph += ("("+str(tmp_count+j+1)+") \'" + Name[node]+"\']}\n")
            node2index[node]=str(tmp_count+j+1)
        tmp_count+=len(graph_ACTOR)
        
    if len(graph_KEYWORD) >0:
        count += 1
        input_graph += " KEYWORD: ["
        for j,node in enumerate(graph_KEYWORD):
            if j != (len(graph_KEYWORD)-1): input_graph += ("("+str(tmp_count+j+1)+") \'" + Name[node]+"\', ")
            else: input_graph += ("("+str(tmp_count+j+1)+") \'" + Name[node]+"\']}\n")
            node2index[node]=str(tmp_count+j+1)
        tmp_count+=len(graph_KEYWORD)

    Input += input_graph
    
    #---------------------------------------------------------------------
    # 构造 Neighborhood information
    input_neighborhood = "One-hop neighbors: {"
    count = 0
    for j,node in enumerate(Neighs):
        if j!= len(Neighs)-1: input_neighborhood += ("("+node2index[node] + "), ")
        else: input_neighborhood += ("("+node2index[node] + ")}\n")
    Input += input_neighborhood
    
    #---------------------------------------------------------------------
    # 构造 RW
    
    if len(RWs)>0:
        input_rw = "Random walks: {"
        for j,path in enumerate(RWs):
            input_rw += (chr(ord('A')+j) + ". ")
            for k,node in enumerate(path):
                if k != len(path)-1:
                    temp = "("+ node2index[node] + ")->"
                    input_rw += temp
                else: 
                    temp = "("+ node2index[node] + ")"
                    input_rw += temp
            if j!= len(RWs)-1: input_rw += "; "
            else: input_rw += "}"
        Input += input_rw
    return Input


In [ ]:
type_text = "{1. Romance 2. Thriller 3. Comedy 4. Action 5. Drama}"
NC_Type = ["Romance", "Thriller", "Comedy", "Action", "Drama"]

In [ ]:
texts = []
for i, central_node in enumerate(train_nodes):
    text = {}
    Instruction = "Given the target MOVIE with the compact graph description in IMDB dataset, what the following categories does this MOVIE belong to: " + type_text + \
                  ". Directly give the answer of this MOVIE‘s categories. This MOVIE may have one or more categories."
    Output = ""
    for l in range(len(Label[central_node])):
        if l != len(Label[central_node]) - 1:
            Output += str(int(Label[central_node][l]) + 1) + ". " + NC_Type[int(Label[central_node][l])] + ", "
        else:
            Output += str(int(Label[central_node][l]) + 1) + ". " + NC_Type[int(Label[central_node][l])]

    Input = get_subgraph(central_node, Neighs[central_node], RWs[central_node])

    text["instruction"] = Instruction
    text["input"] = Input
    text["output"] = Output
    text["index"] = int(central_node)

    texts.append(text)


In [ ]:
import json
import random
# random.shuffle(texts)
with open("nc_imdb.json", "w") as f:
    json.dump(texts[:], f, indent=2)
